In [ ]:
import sys, warnings
sys.path.append('..')
warnings.filterwarnings('ignore')

from pathlib import Path
from dataclasses import dataclass

import numpy as np
from PIL import Image
from tqdm import trange

import torch
from diffusers import StableDiffusionImg2ImgPipeline

from arch import utils, poison

In [ ]:
DEVICE = utils.device_mapper()
print(f"Device: {str(DEVICE).upper()}")

In [ ]:
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", variant='fp16', torch_dtype=torch.float16)
pipeline = pipeline.to(DEVICE)

In [ ]:
PATH = Path('data/puppy.png')
init_image = utils.load_image(PATH)
init_image

---

In [ ]:
def transform(img: Image.Image) -> torch.Tensor:
    # Resize to factor of 32
    w, h = map(lambda x: x - x % 32, img.size)
    img = img.resize((w, h), resample=Image.LANCZOS)

    img = utils.image2array(img, dtype=np.float16)
    img = img * 2.0 - 1.0
    img = img[None].transpose(0, 3, 1, 2)
    return torch.from_numpy(img)

In [ ]:
x = transform(init_image).to(DEVICE)

# adv_x = poison.fgsm(x, model=pipeline.vae.encode)
# adv_x = poison.bim(x, model=pipeline.vae.encode)
adv_x = poison.pgd(x, model=pipeline.vae.encode, eps=0.06, step_size=0.02, iters=1000)

In [ ]:
adv_image = (adv_x / 2 + 0.5).clamp(0, 1)
adv_image = adv_image.cpu().detach().numpy().transpose(0, 2, 3, 1)[0]
adv_image = utils.array2image(adv_image)
adv_image

---

In [ ]:
@dataclass
class HyperConfig:
    prompt = "a photograph of a dog under heavy rain on muddy ground"
    strength = 0.5
    cfg_scale = 7.0
    infer_steps = 50

config = HyperConfig()

In [ ]:
image_nat = pipeline(prompt=config.prompt, 
                        image=init_image, 
                        strength=config.strength, 
                        guidance_scale=config.cfg_scale, 
                        num_inference_steps=config.infer_steps).images[0]

image_adv = pipeline(prompt=config.prompt, 
                        image=adv_image, 
                        strength=config.strength, 
                        guidance_scale=config.cfg_scale, 
                        num_inference_steps=config.infer_steps).images[0]

In [ ]:
labels = ['Source Image', 'Adv Image', 'Gen. Image Nat.', 'Gen. Image Adv.']
data = [init_image, adv_image, image_nat, image_adv]

utils.contact_layer(data, 1, 4, labels)